# CMSTP Execution
Detects various indicators of Microsoft Connection Manager Profile Installer execution

## Rule Content
```
- action: global
  title: CMSTP Execution
  id: 9d26fede-b526-4413-b069-6e24b6d07167
  status: stable
  description: Detects various indicators of Microsoft Connection Manager Profile
    Installer execution
  tags:
  - attack.defense_evasion
  - attack.execution
  - attack.t1191
  - attack.g0069
  - car.2019-04-001
  author: Nik Seetharaman
  references:
  - http://www.endurant.io/cmstp/detecting-cmstp-enabled-code-execution-and-uac-bypass-with-sysmon/
  detection:
    condition: 1 of them
  fields:
  - CommandLine
  - ParentCommandLine
  - Details
  falsepositives:
  - Legitimate CMSTP use (unlikely in modern enterprise environments)
  level: high
- logsource:
    product: windows
    service: sysmon
  detection:
    selection2:
      EventID: 12
      TargetObject: '*\cmmgr32.exe*'
    selection3:
      EventID: 13
      TargetObject: '*\cmmgr32.exe*'
    selection4:
      EventID: 10
      CallTrace: '*cmlua.dll*'
- logsource:
    category: process_creation
    product: windows
  detection:
    selection1:
      ParentImage: '*\cmstp.exe'

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"12" AND registry_key_path.keyword:*\\cmmgr32.exe*) OR (event_id:"13" AND registry_key_path.keyword:*\\cmmgr32.exe*) OR (event_id:"10" AND process_call_trace.keyword:*cmlua.dll*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='process_parent_path.keyword:*\\cmstp.exe')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()